# Bioinformatics Final Project: Group 4
### Coral Host taken into consideration is OANN.

In [2]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import numpy as np

Loading the Gene Expression data and the meta data files for the Analysis.

In [3]:
# Loading the Gene Expression Data
Gene_expression = pd.read_csv('normalized_counts_vst.csv', index_col = 0).T
Gene_expression.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,S0EMV0,S8GJB7,V6CLA2,W0LYS5,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2
M_c1_C,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,1.620841,1.620841,1.856850,3.104000,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284
M_c3_C,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,1.620841,1.869137,2.976699,3.426356,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146
M_c5_C,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,1.948261,2.346861,3.570151,2.414528,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485
M_c6_C,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,1.620841,2.544092,2.905994,2.544092,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715
M_c7_C,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,1.887821,2.152548,3.050253,2.454656,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568


In [4]:
# Loading the metadata file.
metadata = pd.read_csv("metadata.csv")
metadata.head(5)

,Sample_ID,Clade,Host,Dominant
0,M_c1_C,C,MCAV,Yes
1,M_c3_C,C,MCAV,Yes
2,M_c5_C,C,MCAV,Yes
3,M_c6_C,C,MCAV,No
4,M_c7_C,C,MCAV,Yes


Merging the data.

In [5]:
# Merging the Gene Expression and Meta data together.
data = pd.merge(Gene_expression, metadata, left_index = True, right_on = 'Sample_ID')
data.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2,Sample_ID,Clade,Host,Dominant
0,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284,M_c1_C,C,MCAV,Yes
1,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146,M_c3_C,C,MCAV,Yes
2,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485,M_c5_C,C,MCAV,Yes
3,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715,M_c6_C,C,MCAV,No
4,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568,M_c7_C,C,MCAV,Yes


In [6]:
# Dropping the column that are not taken into consideration.
data = data.drop(['Sample_ID','Clade', 'Dominant' ],axis = 1)
data.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,S8GJB7,V6CLA2,W0LYS5,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2,Host
0,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,1.620841,1.856850,3.104000,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284,MCAV
1,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,1.869137,2.976699,3.426356,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146,MCAV
2,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,2.346861,3.570151,2.414528,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485,MCAV
3,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,2.544092,2.905994,2.544092,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715,MCAV
4,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,2.152548,3.050253,2.454656,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568,MCAV


In [7]:
# Making the Coral HOST into numericals by making HOST == OANN as 1 and HOST != OANN as 0.
data['Host'] = (data['Host'] == 'OANN').astype(int)
data.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,S8GJB7,V6CLA2,W0LYS5,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2,Host
0,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,1.620841,1.856850,3.104000,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284,0
1,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,1.869137,2.976699,3.426356,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146,0
2,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,2.346861,3.570151,2.414528,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485,0
3,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,2.544092,2.905994,2.544092,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715,0
4,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,2.152548,3.050253,2.454656,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568,0


The HOST is our Target and the rest of the data is the inputs.

In [8]:
# Target column
y = data['Host']
# Inputs of the target column
X = data.iloc[:, :-1]

In [9]:
# Splitting the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 20)

## Logistic Regression

### Cross validation results

In [10]:
# Cross Validation 
# Defining the cross validation accuracy.
lr = LogisticRegression(max_iter = 1000)

# Cross Validation Accuracy with a fold of 5.
cv_score = cross_val_score(lr, X_train, y_train, cv = 5, scoring = 'accuracy')
print("Logistic Regression CV Accuracy:", cv_score.mean())

Logistic Regression CV Accuracy: 0.9714285714285715


### Fitting the model with all parameters and testing the accuracy on the test set.

In [11]:
# FItting the model.
lr.fit(X_train, y_train)

# Predicting the results on the testing set.4
y_pred = lr.predict(X_test)

# FInding the Accurayc of the logistic Reqgression Model.
Accuracy = accuracy_score(y_test, y_pred)

print("Logistic Regression Testing Accuracy:", Accuracy)

Logistic Regression Testing Accuracy: 0.8888888888888888


### Creating the confusion Matrix of the predictions and finding the Error rate fo the Predictions.

In [12]:
# Assuming y_test and y_pred are already defined
cons_matrix = confusion_matrix(y_test, y_pred)

# Calculate the error rate
Error_rate = (cons_matrix[0, 1] + cons_matrix[1, 0]) / cons_matrix.sum()

# Prepare table data
table_data = [
    ["Actual Positive", cons_matrix[0, 0], cons_matrix[0, 1]],
    ["Actual Negative", cons_matrix[1, 0], cons_matrix[1, 1]]
]

# Defining the headers
headers = ["Confusion Matrix:", "Predicted Positive", "Predicted Negative"]

print(tabulate(table_data, headers, tablefmt = "grid"))
print(f"Error Rate: {Error_rate:.4f}")

+---------------------+----------------------+----------------------+
| Confusion Matrix:   |   Predicted Positive |   Predicted Negative |
+=====================+======================+======================+
| Actual Positive     |                    7 |                    0 |
+---------------------+----------------------+----------------------+
| Actual Negative     |                    1 |                    1 |
+---------------------+----------------------+----------------------+
Error Rate: 0.1111


### Finding the feature importance for the logistic regression model.

In [13]:
# Get the coefficients and corresponding feature names
coefficients = lr.coef_[0]
feature_names = X_train.columns

# Create a DataFrame to display coefficients and feature names
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Display the DataFrame sorted by absolute coefficient values
coefficients_df['Absolute Coefficient'] = coefficients_df['Coefficient'].abs()
sorted_coefficients = coefficients_df.sort_values(by='Absolute Coefficient', ascending = False)

# Calculate the mean and standard deviation of the absolute coefficients
mean_coeff = np.mean(sorted_coefficients['Absolute Coefficient'].abs())
std_coeff = np.std(sorted_coefficients['Absolute Coefficient'].abs())

# Setting the threshold
threshold = mean_coeff + std_coeff

# Select features with absolute coefficients greater than the threshold
significant_features = sorted_coefficients[sorted_coefficients['Absolute Coefficient'].abs() > threshold]
print(f"The number of significant features required for the resutls within in top 32% important features : {len(significant_features)}")

# Setting the threshold
threshold = mean_coeff + std_coeff*2
# Select features with absolute coefficients greater than the threshold
significant_features = sorted_coefficients[sorted_coefficients['Absolute Coefficient'].abs() > threshold]
print(f"The number of significant features required for the resutls within in top 5% important features : {len(significant_features)}")

# Top 10 important features
print("Top 10 features contributing to the results:-")
top_features = sorted_coefficients['Feature'].head(10)
print(top_features.to_list())

The number of significant features required for the resutls within in top 32% important features : 458
The number of significant features required for the resutls within in top 5% important features : 179
Top 10 features contributing to the results:-
['Q8ZRS8', 'P46285', 'Q43011', 'Q9SF16', 'Q0V6P9', 'Q53RK8', 'Q94B38', 'Q0WKV3', 'Q28057', 'Q3ZBH0']


### Hyperparameter Tunning for Logistic Regression.

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.exceptions import FitFailedWarning

warnings.filterwarnings('ignore', category=FitFailedWarning)
warnings.filterwarnings('ignore')

# Define the parameter grid for hyperparameter tuning
param_grid = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga'], 'max_iter': [100, 200, 300]},
    {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['elasticnet'], 'solver': ['saga'], 'max_iter': [100, 200, 300]}
]


# Create a Logistic Regression model
logreg = LogisticRegression()

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Output the best parameters and the corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'C': 0.1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score: 1.0


### Fitting the model with the best parameters and predicting the results

In [15]:
# Creating the logistic regression object.
lr = LogisticRegression(C= 0.01, max_iter= 1000, penalty= 'l2', solver= 'liblinear')

# FItting the model.
lr.fit(X_train, y_train)

# Predicting the results on the testing set.4
y_pred = lr.predict(X_test)

# FInding the Accurayc of the logistic Reqgression Model.
Accuracy = accuracy_score(y_test, y_pred)

print("Logistic Regression Testing Accuracy:", Accuracy)

Logistic Regression Testing Accuracy: 0.8888888888888888



## Questions 

### 1) How Many Genes/Components Are Obtained/Required Significant Results? 
  In this question, based on your algorithm, are there a certain number of genes or components (depending on the algorithm) that can perform the classification of your problem? 

- The number of significant features required for the resutls within in top 32% important features : 458
- The number of significant features required for the resutls within in top 5% important features : 179
- Top 10 features contributing to the results can do the classification:-
- ['Q8ZRS8', 'P46285', 'Q43011', 'Q9SF16', 'Q0V6P9', 'Q53RK8', 'Q94B38', 'Q0WKV3', 'Q28057', 'Q3ZBH0']

### 2) What is the Error Rate of These Algorithms? 
  In this question, based on your algorithms, how much of the test dataset is misclassified?

- Error Rate is 0.111 for the test dataset. Only 1 is misclassified.
  
### 3) What are two genes that were signficiantly correlated to your problem? 
  In this question, based on your algorithm, identify and annotate 2 genes using [uniprot](uniprot.org). Do you think they are biologically relevant? 

- Aconitate Hydratase B (AcnB) from Salmonella typhimurium (Q8ZRS8 · ACNB_SALTY):

Function: Involved in the catabolism of short-chain fatty acids via the tricarboxylic acid (TCA) cycle and the 2-methylcitrate cycle. It catalyzes the reversible isomerization of citrate to isocitrate and the hydration of 2-methyl-cis-aconitate.
Biological Relevance: The TCA cycle is a key metabolic pathway that provides energy and precursors for biosynthesis. In the context of Cladacopium, this enzyme could be crucial for energy metabolism. The ability to process various carbon sources, including short-chain fatty acids, might be significant in the symbiotic relationship, possibly affecting the nutrient exchange dynamics.

- Sedoheptulose-1,7-bisphosphatase, Chloroplastic from Triticum aestivum (Wheat) (P46285 · S17P_WHEAT):

Function: This enzyme plays a role in the Calvin cycle, which is involved in photosynthesis.
Biological Relevance: In relation to Cladacopium, which are photosynthetic algae living in coral tissues, this enzyme could be significant for photosynthesis and carbon fixation. Its role in the Calvin cycle implies its importance in converting carbon dioxide into organic compounds, which is vital for the symbiotic relationship in providing nutrients to the coral host.

- Given their functions in crucial metabolic pathways, both enzymes are likely to be biologically relevant in the context of Cladacopium, especially considering the metabolic and photosynthetic processes essential for the symbiosis with coral hosts. 

### 4) What can we infer about this dataset based on these results? 
  In this question, you should think about the overall classification results of this algorithm. Does the type of algorithm used here tell you anything about the data in this study? What is biologically meaningful about this? 

- From the results 88% of accuracy on test data, 97.14% Cross validation accuracy and the predictions of Logistic Regression algorithm used can provide insights into whether the focus is on expression patterns, functional annotations, or evolutionary relationships, each offering different perspectives on the biological significance of the data. 

# Linear Discriminant Analysis

### Performing cross validation on the dataset

In [16]:
# Initializing the LDA model
lda = LinearDiscriminantAnalysis()

# Performing a  5-fold cross-validation on the training data to evaluate the model's performance
cv_score = cross_val_score(lda, X_train, y_train, cv=5, scoring='accuracy')

# Print the mean cross-validated accuracy score
print("Linear Discriminant Analysis Cross-validated Accuracy:", cv_score.mean())

Linear Discriminant Analysis Cross-validated Accuracy: 0.9428571428571428


### Fitting the model and predicting the results on the test dataset.

In [17]:
# Fitting the LDA model to the training data
lda.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = lda.predict(X_test)

# Calculate and print the accuracy score on the test set
# 'y_test' are the true labels for the test data
test_accuracy = accuracy_score(y_test, y_pred)
print("Linear Discriminant Analysis Test data Accuracy:", test_accuracy)

Linear Discriminant Analysis Test data Accuracy: 0.8888888888888888


### Finding different matrices of evaluation results.

In [18]:
# Calculating the decision function value for each sample in the test set.
# This value indicates the distance of each sample from the decision boundary.
lda_DV = lda.decision_function(X_test)
print("\nDecision Values:\n", lda_DV)

# Calculating the probability of each class for each sample in the test set.
# These probabilities show the model's confidence in assigning each class to each sample.
lda_PP = lda.predict_proba(X_test)
print("\nPredicted Probabilities:\n", lda_PP)

# Predicting the class labels for the test set.
# These are the final classifications made by the model for each sample.
lda_PC = lda.predict(X_test)
print("\nPredicted Classes:\n", lda_PC)

# Generating a classification report that includes key metrics like precision, recall, and F1-score.
# This report provides a detailed analysis of the model's performance for each class.
lda_CR = classification_report(y_test, y_pred)
print("\nClassification Report:\n", lda_CR)



Decision Values:
 [-22.51474059 -21.3001699   -5.5964112  -24.98414154 -33.95100689
 -24.87532512  21.31513501 -16.56494812 -23.70147123]

Predicted Probabilities:
 [[1.00000000e+00 1.66714126e-10]
 [9.99999999e-01 5.61634463e-10]
 [9.96302563e-01 3.69743668e-03]
 [1.00000000e+00 1.41099409e-11]
 [1.00000000e+00 1.79996912e-15]
 [1.00000000e+00 1.57319865e-11]
 [5.53292079e-10 9.99999999e-01]
 [9.99999936e-01 6.39638226e-08]
 [1.00000000e+00 5.08840687e-11]]

Predicted Classes:
 [0 0 0 0 0 0 1 0 0]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93         7
           1       1.00      0.50      0.67         2

    accuracy                           0.89         9
   macro avg       0.94      0.75      0.80         9
weighted avg       0.90      0.89      0.87         9



### Confustion Matrix of the LDA Analysis

In [19]:
# Assuming y_test and y_pred are already defined
cons_matrix = confusion_matrix(y_test, y_pred)

# Calculate the error rate
Error_rate = (cons_matrix[0, 1] + cons_matrix[1, 0]) / cons_matrix.sum()

# Prepare table data
table_data = [
    ["Actual Positive", cons_matrix[0, 0], cons_matrix[0, 1]],
    ["Actual Negative", cons_matrix[1, 0], cons_matrix[1, 1]]
]

# Defining the headers
headers = ["Confusion Matrix:", "Predicted Positive", "Predicted Negative"]

print(tabulate(table_data, headers, tablefmt = "grid"))
print(f"Error Rate: {Error_rate:.4f}")

+---------------------+----------------------+----------------------+
| Confusion Matrix:   |   Predicted Positive |   Predicted Negative |
+=====================+======================+======================+
| Actual Positive     |                    7 |                    0 |
+---------------------+----------------------+----------------------+
| Actual Negative     |                    1 |                    1 |
+---------------------+----------------------+----------------------+
Error Rate: 0.1111


### Recursive Feature Elimination (RFE)
- RFE is a technique for selecting features by recursively considering smaller and smaller sets of features.

In [20]:
from sklearn.feature_selection import RFE

# Initialize LDA model
lda = LinearDiscriminantAnalysis()

# Creating a list of the features using the columns of X
features = X.columns.tolist()

# Initialize RFE with the LDA model and the desired number of features
rfe = RFE(lda, n_features_to_select = 40)

# Fit RFE
rfe.fit(X, y)

# Get the ranking of the features
feature_ranking = rfe.ranking_

# Select the top features based on ranking
top_features = [features[i] for i in range(len(features)) if feature_ranking[i] == 1]
print(top_features)

['A0A075HNX4', 'A5FKJ2', 'B4GUZ2', 'B7GJ95', 'O30124', 'P07335', 'P22694', 'P32019', 'P32811', 'P51574', 'P9WKM1', 'Q0J9V3', 'Q23551', 'Q54FZ8', 'Q54GE3', 'Q54HL8', 'Q5M9G6', 'Q5RCS8', 'Q5UPD4', 'Q5YLM1', 'Q5ZKR7', 'Q68FR8', 'Q6GNI4', 'Q6ICB0', 'Q6P9J9', 'Q6S5H5', 'Q7RJG2', 'Q80ZA4', 'Q86WI1', 'Q86XH1', 'Q8GYH8', 'Q8L611', 'Q8L716', 'Q949Y0', 'Q9BSJ2', 'Q9BXP8', 'Q9EPK2', 'Q9HDZ5', 'Q9M271', 'Q9UJZ1']


### Finding the most influential features using the coefficient method.

In [21]:
# Creating the LDA object and fitting the model.
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Get the absolute coefficients for each gene
coefficients = abs(lda.coef_[0])  # Assuming binary classification for simplicity

# Combine gene names with their coefficients
gene_importance = list(zip(features, coefficients))

# Sorting the genes by their importance
sorted_genes = sorted(gene_importance, key=lambda x: x[1], reverse=True)

# Display the most influential genes
print("Most influential genes:", sorted_genes[:10])

Most influential genes: [('Q54GE3', 0.32893503837190824), ('B9F676', 0.3269164164157803), ('P54399', 0.30848877233608185), ('Q0VCN3', 0.2829772938120041), ('Q9HDZ5', 0.22735039343467034), ('Q96S16', 0.21421552854237969), ('A3LNF8', 0.20495138631202564), ('A1CUK5', 0.20481453412744383), ('O94264', 0.20306847313909698), ('Q55G18', 0.19048664760273734)]


### Finding the number of genes that are significant to the results.

In [22]:
# Calculate the mean and standard deviation of the absolute coefficients
mean_coeff = np.mean(coefficients)
std_coeff = np.std(coefficients)

# Setting the threshold
threshold = mean_coeff + std_coeff

# Extract genes with importance score above the threshold
genes_above_threshold = [gene[0] for gene in sorted_genes if gene[1] > threshold]

# Selecting features with absolute coefficients greater than the threshold
print(f"The number of significant genes present for the resutls within in top 32% important features : {len(genes_above_threshold)}")

# Setting the threshold
threshold = mean_coeff + std_coeff*2
# Selecting features with absolute coefficients greater than the threshold
genes_above_threshold = [gene[0] for gene in sorted_genes if gene[1] > threshold]
print(f"The number of significant genes present for the resutls within in top 5% important features : {len(genes_above_threshold)}")
print("Top 10 features :", genes_above_threshold[:10])


The number of significant genes present for the resutls within in top 32% important features : 310
The number of significant genes present for the resutls within in top 5% important features : 105
Top 10 features : ['Q54GE3', 'B9F676', 'P54399', 'Q0VCN3', 'Q9HDZ5', 'Q96S16', 'A3LNF8', 'A1CUK5', 'O94264', 'Q55G18']


- Q54GE3 and Q9HDZ5 are the two genes that are common in the most influential features and the features that are found by Recursive Feature Elimination.

## Questions 

### How Many Genes/Components Are Obtained/Required Significant Results? 
  In this question, based on your algorithm, are there a certain number of genes or components (depending on the algorithm) that can perform the classification of your problem? 

- The number of significant genes present for the resutls within in top 32% important features : 310
- The number of significant genes present for the resutls within in top 5% important features : 105
- Top 10 features that contribute to the classification are:- 
- ['Q54GE3', 'B9F676', 'P54399', 'Q0VCN3', 'Q9HDZ5', 'Q96S16', 'A3LNF8', 'A1CUK5', 'O94264', 'Q55G18']

### What is the Error Rate of These Algorithms? 
  In this question, based on your algorithms, how much of the test dataset is misclassified?  
- The Error Rate is similar to the results of the Logistic Regression. Only 1 is misclassified.
  
### What are two genes that were signficiantly correlated to your problem? 
  In this question, based on your algorithm, identify and annotate 2 genes using [uniprot](uniprot.org). Do you think they are biologically relevant? 

- Q54GE3 · VPS45_DICDI: 
- Involved in vesicle-mediated protein trafficking, particularly in the Golgi stack and trans-Golgi network. This process is crucial for cellular functioning, as it is involved in sorting and transporting proteins to their destination.

- Q9HDZ5 · YKP9_SCHPO:
- CRAL-TRIO domain-containing protein C589.09: Predicted to be mitochondrial, indicating a role in cellular energy metabolism or other mitochondrial functions. - - The CRAL-TRIO domain is often involved in binding small lipophilic molecules, suggesting a potential role in lipid metabolism or signaling.

- While the direct relevance of these specific proteins to Cladacopium and its coral host might not be immediately clear, the functional roles they play in fundamental cellular processes could offer indirect insights into the biological mechanisms underlying symbiosis. We are unclear about the biologically direct relationship.

### What can we infer about this dataset based on these results? 
  In this question, you should think about the overall classification results of this algorithm. Does the type of algorithm used here tell you anything about the data in this study? What is biologically meaningful about this?

- The overall classification was similar to the results of the Logistic Regression with an Accuracy of 88% but the Cross validation accuracy of 94.28% is less than Logistic Regression. The identification of VPS45 and CRAL-TRIO domain-containing protein C589.09, and the use of LDA, suggest a focus on distinguishing cellular states or types based on fundamental biological processes.

# Support Vector Machine (SVM)

### Cross-validation score with SVM.

In [23]:
# Initializing the Support Vector Machine classifier with linear kernel and degree 5
svm = SVC(kernel='linear', degree=5)

# Performing 5-fold cross-validation to estimate the accuracy of the SVM
cv_scores = cross_val_score(svm, X_train, y_train, cv=5, scoring='accuracy')
print("Support Vector Machine Cross-validated Accuracy:", cv_scores.mean())

Support Vector Machine Cross-validated Accuracy: 0.9714285714285715


### Fitting the Model using SVM.

In [24]:
# Fitting the SVM model to the training data
svm.fit(X_train, y_train)

# Retrieving the indices of the support vectors
support_vector_indices = svm.support_

# Extracting the gene names corresponding to the support vector indices
selected_genes = X_train.columns[support_vector_indices]
print("Genes Used for Prediction:\n", selected_genes.to_list())

Genes Used for Prediction:
 ['A0A023I7E1', 'A0A067XMP1', 'A0A0A0RM07', 'A0A0G2JDV3', 'A0A0G2JEB6', 'A0A0H2VDN9', 'A0A0I9QGZ2', 'A0A0K9RDW0', 'A0A0R4IBK5', 'A0A125YZN2', 'A0A1B4XBG9', 'A0A1D8PDZ1', 'A0A4V8H042', 'A0A075HNX4', 'A0A0B4KGY6', 'A0A0B6VQ48']


### Evaluation using the Test data

In [25]:
# Predicting the labels for the test set
y_pred = svm.predict(X_test)

# Calculating the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print("Support Vector Machine Test Set Accuracy:", test_accuracy)

# Displaying the number of support vectors used in the model
print("Number of Components (Support Vectors):", svm.support_vectors_.shape[0])

Support Vector Machine Test Set Accuracy: 0.8888888888888888
Number of Components (Support Vectors): 16


### Hyperparameter tuning of SVM.

In [26]:
# Hyperparameter tuning for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': [1, 0.1, 0.01, 0.001],  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    'kernel': ['linear', 'rbf', 'poly']
}

# Creating a GridSearchCV object to find the best parameters
grid_search = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv = 5)

# Fitting the grid search to the data
grid_search.fit(X_train, y_train)

# Printing the best parameters found by GridSearchCV
print("Best Parameters:", grid_search.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.857 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.714 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.714 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.833 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.667 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.667 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.857 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

### Prediction using the best Parameters from teh Hyperparameter tuning.

In [27]:
# Predicting with the best estimator
y_pred_tuned = grid_search.predict(X_test)

# Calculating the accuracy on the test set with the tuned parameters
tuned_accuracy = accuracy_score(y_test, y_pred_tuned)
print("Tuned SVM Test Set Accuracy:", tuned_accuracy)

Tuned SVM Test Set Accuracy: 0.8888888888888888


### Confusion Matrix for SVM.

In [28]:
# Assuming y_test and y_pred are already defined
cons_matrix = confusion_matrix(y_test, y_pred)

# Calculate the error rate
Error_rate = (cons_matrix[0, 1] + cons_matrix[1, 0]) / cons_matrix.sum()

# Prepare table data
table_data = [
    ["Actual Positive", cons_matrix[0, 0], cons_matrix[0, 1]],
    ["Actual Negative", cons_matrix[1, 0], cons_matrix[1, 1]]
]

# Defining the headers
headers = ["Confusion Matrix:", "Predicted Positive", "Predicted Negative"]

print(tabulate(table_data, headers, tablefmt = "grid"))
print(f"Error Rate: {Error_rate:.4f}")

+---------------------+----------------------+----------------------+
| Confusion Matrix:   |   Predicted Positive |   Predicted Negative |
+=====================+======================+======================+
| Actual Positive     |                    7 |                    0 |
+---------------------+----------------------+----------------------+
| Actual Negative     |                    1 |                    1 |
+---------------------+----------------------+----------------------+
Error Rate: 0.1111


### Identifying the most significant genes

In [31]:
# Finding the coefficients of SVM.
coefficients = svm.coef_[0]

# Ranking the genes by the absolute values of the coefficients
gene_importance = sorted(zip(X_train.columns, coefficients), key=lambda x: abs(x[1]), reverse=True)

# Selecting the top 2 significant genes
top_genes = gene_importance[:2]
print("Top 2 Significant Genes:\n", top_genes)

# Annotate these genes
for gene, weight in top_genes:
    # Perform a literature search or database query for each gene
    # Annotate the biological relevance based on your findings
    print(f"Gene: {gene}, Weight: {weight}")


Top 2 Significant Genes:
 [('Q8ZRS8', 0.003722035964487068), ('Q9SF16', 0.0036126978014409186)]
Gene: Q8ZRS8, Weight: 0.003722035964487068
Gene: Q9SF16, Weight: 0.0036126978014409186


## Questions 

### How Many Genes/Components Are Obtained/Required Significant Results? 
  In this question, based on your algorithm, are there a certain number of genes or components (depending on the algorithm) that can perform the classification of your problem? 

- For the Support Vector machines 16 components can perform the classification of your problem.
- The genes used for Prediction are 'A0A023I7E1', 'A0A067XMP1', 'A0A0A0RM07', 'A0A0G2JDV3', 'A0A0G2JEB6', 'A0A0H2VDN9', 'A0A0I9QGZ2', 'A0A0K9RDW0', 'A0A0R4IBK5', 'A0A125YZN2', 'A0A1B4XBG9', 'A0A1D8PDZ1', 'A0A4V8H042', 'A0A075HNX4', 'A0A0B4KGY6', 'A0A0B6VQ48'

### What is the Error Rate of These Algorithms? 
  In this question, based on your algorithms, how much of the test dataset is misclassified? 

- The Error Rate of the SVM is 0.1111 and only 1 value is misclassified.

### What are two genes that were signficiantly correlated to your problem? 
  In this question, based on your algorithm, identify and annotate 2 genes using [uniprot](uniprot.org). Do you think they are biologically relevant? 

T-complex Protein 1 Subunit Eta (CCT7) - Arabidopsis thaliana ( Q9SF16 · TCPH_ARATH ):

- Function: Acts as a molecular chaperone, assisting in the folding of proteins upon ATP hydrolysis. It plays a role in the folding of actin and tubulin, which are crucial for cellular structure and dynamics.
- Relevance to Cladacopium: In the context of Cladacopium, protein folding and structural integrity are essential for maintaining cellular function, especially under stress conditions prevalent in symbiotic environments. Molecular chaperones like CCT7 might be crucial in ensuring the proper functioning of proteins that sustain the photosynthetic and metabolic activities of the algae.

Aconitate Hydratase B (AcnB) - Salmonella typhimurium ( Q8ZRS8 · ACNB_SALTY ):

- Function: Involved in the catabolism of short-chain fatty acids through the TCA cycle and the 2-methylcitrate cycle. It plays a role in isomerizing citrate to isocitrate and in the hydration of 2-methyl-cis-aconitate.
- Relevance to Cladacopium: The TCA cycle is fundamental for energy metabolism. In Cladacopium, efficient energy metabolism is crucial for sustaining the symbiotic relationship with coral hosts. The ability to process various carbon sources might impact the nutrient exchange and energy dynamics within the symbiosis.

### What can we infer about this dataset based on these results? 
  In this question, you should think about the overall classification results of this algorithm. Does the type of algorithm used here tell you anything about the data in this study? What is biologically meaningful about this? 
The Support vector machines are able to perform with an Cross validation accuracy of 97.14% which is good. The fact that these proteins Q8ZRS8 · ACNB_SALTY and Q9SF16 · TCPH_ARATH were highlighted as significant features in an SVM analysis suggests they might have distinctive roles or expressions in certain conditions or states.


# Partial 

In [ ]:
pls = PLSRegression(n_components=5) 
y_pred_cv = cross_val_predict(pls, X_train, y_train, cv=5)
threshold = 0.5 
y_pred_cv = (y_pred_cv > threshold).astype(int)
cv_accuracy = accuracy_score(y_train, y_pred_cv)
print("Partial Least Square Cross-validated Accuracy:", cv_accuracy)
pls.fit(X_train, y_train)
y_pred = pls.predict(X_test)
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)
test_accuracy = accuracy_score(y_test, y_pred_binary)
print("Partial Least Square Test Set Accuracy:", test_accuracy)
error_rate = 1 - test_accuracy
print(f"Error Rate for Partial Least Square : {error_rate}")
loadings_component_1 = pls.x_weights_[:, 0]
top_gene_indices = loadings_component_1.argsort()[-5:][::-1]
top_genes = X_train.columns[top_gene_indices]
print("Top Genes for PLS Method: ", top_genes)
conf_matrix = confusion_matrix(y_test, y_pred_binary)